  CÓDIGO DE AUTOMATIZACIÓN DE LA BBDD DE SQM  (ITERA SOBRE SISTEMAS Y SUBSISTEMAS Y PUNTOS DE MONITOREO)

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import os
import time

def configure_driver(download_folder):
    options = Options()
    prefs = {
        "download.default_directory": download_folder,
        "download.prompt_for_download": False,
        "download.directory_upgrade": True,
        "safebrowsing.enabled": True
    }
    options.add_experimental_option("prefs", prefs)
    driver = webdriver.Chrome(options=options)
    return driver

base_download_path = "C:\\Users\\fdona\\OneDrive\\Escritorio\\DESCARGAS_SQM"

try:
    temp_driver = webdriver.Chrome()
    temp_driver.get("https://www.sqmsenlinea.com/env-systems?type=followup")
    WebDriverWait(temp_driver, 10).until(
        EC.visibility_of_element_located((By.CSS_SELECTOR, "select[name='data_source_type_id']")))
    Select(temp_driver.find_element(By.CSS_SELECTOR, "select[name='data_source_type_id']")).select_by_visible_text("Niveles")
    select_sistema = Select(WebDriverWait(temp_driver, 10).until(
        EC.element_to_be_clickable((By.NAME, "environmental_system_id"))))
    sistemas = [option.text for option in select_sistema.options[1:]]
    temp_driver.quit()
    

    for sistema in sistemas:
        sistema_folder = os.path.join(base_download_path, sistema)
        if not os.path.exists(sistema_folder):
            os.makedirs(sistema_folder)
        
        driver = configure_driver(sistema_folder)
        driver.get("https://www.sqmsenlinea.com/env-systems?type=followup")
        # SELECCIONANDO NIVELES
        Select(WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.CSS_SELECTOR, "select[name='data_source_type_id']")))).select_by_visible_text("Niveles")
        Select(WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.NAME, "environmental_system_id")))).select_by_visible_text(sistema)
        time.sleep(5)
        # SELECCIONANDO SUBSISTEMAS
        select_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.NAME, "children_id")))
        select_subsistema = Select(select_element)
        # COMPROBANDO SI HAY SUBSISTEMAS
        if len(select_subsistema.options) > 1:
            for index_subsistema in range(1, len(select_subsistema.options)):
                print(f"Manejando subsistema con índice {index_subsistema}")
                select_subsistema.select_by_index(index_subsistema)
                time.sleep(3)
                
        else:
            print("No hay subsistemas. Manejando puntos de monitoreo directamente.")
            select_element = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.NAME, "infrastructure_id")))
            select = Select(select_element)
            for elements in range(1, min(3, len(select.options))):
                select.select_by_index(elements)
                time.sleep(5)
                # CLICK EN DESCARGA
                WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "export-btn-niveles-manuales"))).click()
                time.sleep(10)
                # VUELVE A DEJAR DISPO LA LISTA DESPLEGABLE DE PTOS 
                select_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.NAME, "infrastructure_id")))
                select = Select(select_element)
                options = select.options
                
        driver.quit()
        

finally:
    print("¡¡¡Proceso completado...!!!")


CONNECCION A BBDD POSTGREE SQL


In [ ]:
import psycopg2

try:
    # Parámetros de conexión
    connection = psycopg2.connect(
        user="tu_usuario",
        password="tu_contraseña",
        host="tu_host",
        port="tu_puerto",
        database="tu_basededatos"
    )

    # Crear un cursor para ejecutar consultas
    cursor = connection.cursor()

    # Ejecutar una consulta
    cursor.execute("SELECT * FROM TABLA")

  

except (Exception, psycopg2.Error) as error:
    print("Error al conectar a PostgreSQL:", error)

finally:
    # Cerrar la conexión
    if (connection):
        cursor.close()
        connection.close()
        print("Conexión a PostgreSQL cerrada")
